In [50]:
import pandas as pd
import numpy as np
import datetime as date
import os
import tqdm
import ast
import json
import sqlite3
from genderize import Genderize

In [51]:
con = sqlite3.connect("../data/database/ICLR.db")
df_submission_features = pd.read_csv("../data/df_rr_all_features.csv")

In [52]:
#olde dont use
query = '''
SELECT 
    AVG(reviews.rating_int) AS AVG_rating,
    AVG(reviews.confidence_int) AS AVG_confidence,
    AVG(CAST(author_gscholar_citation.cite as demcial)) as AVG_CITE,
    MAX(CAST(author_gscholar_citation.cite as demcial)) as MAX_CITE,
    COUNT(author_submission.author_id) as AUTHOR_COUNT,
    submissions.* 
FROM submissions 
JOIN author_submission ON submissions.id = author_submission.submission_id
JOIN reviews ON submissions.id = reviews.forum
WHERE submissions.conf_year in (2017,2018,2019)
GROUP BY submissions.id'''


In [53]:
query = '''
SELECT 
    AVG(CAST(author_gscholar_citation.cite as demcial)) as AVG_CITE,
    MAX(CAST(author_gscholar_citation.cite as demcial)) as MAX_CITE,
    COUNT(author_submission.author_id) as AUTHOR_COUNT,
    submissions.* 
FROM submissions 
JOIN author_submission ON submissions.id = author_submission.submission_id
LEFT JOIN author_gscholar_citation ON author_submission.author_id = author_gscholar_citation.author_id 
    AND submissions.conf_year = author_gscholar_citation.year
WHERE submissions.conf_year in (2017,2018,2019)
GROUP BY submissions.id'''

df_submission = pd.read_sql_query(query,con)
df_submission

,AVG_CITE,MAX_CITE,AUTHOR_COUNT,id,original,tcdate,tmdate,number,title,authorids,authors,keywords,abstract,pdf,conf_name,conf_year,tldr
0,1523.500000,2994.0,3,B1-Hhnslg,None,1478376505434,1481060777491,581,Prototypical Networks for Few-shot Learning,"[""jsnell@cs.toronto.edu"", ""kswersky@twitter.co...","[""Jake Snell"", ""Kevin Swersky"", ""Richard Zemel""]","[""Deep learning"", ""Transfer Learning""]",A recent approach to few-shot classification c...,/pdf/1e1d8a1884188e3ce0db825def307a25c844c868.pdf,ICLR,2017,We learn a metric space in which few-shot clas...
1,789.000000,1401.0,2,B1-q5Pqxl,None,1478290057546,1489370977828,417,Machine Comprehension Using Match-LSTM and Ans...,"[""shwang.2014@phdis.smu.edu.sg"", ""jingjiang@sm...","[""Shuohang Wang"", ""Jing Jiang""]","[""Natural language processing"", ""Deep learning""]",Machine comprehension of text is an important ...,/pdf/78402508a0a3ebc1c9078ef8ff48dccc6fef6cd8.pdf,ICLR,2017,Using Match-LSTM and Answer Pointer to select ...
2,168.666667,275.0,3,B12Js_yRb,ryiyj_yRW,1509030627915,1518730187825,136,Learning to Count Objects in Natural Images fo...,"[""yz5n12@ecs.soton.ac.uk"", ""jsh2@ecs.soton.ac....","[""Yan Zhang"", ""Jonathon Hare"", ""Adam Pr\u00fcg...","[""visual question answering"", ""vqa"", ""counting""]",Visual Question Answering (VQA) models have st...,/pdf/b0ec608c19a908ac8ccc2d67f8dc370a699ae26c.pdf,ICLR,2018,Enabling Visual Question Answering models to c...
3,14.000000,14.0,1,B13EC5u6W,HJh409uTW,1508580916492,1518730191139,36,Thinking like a machine — generating visual ra...,"[""jarrelscy@gmail.com""]","[""Jarrel Seah"", ""Jennifer Tang"", ""Andy Kitchen...","[""interpretability"", ""generative adversarial n...",Interpretability and small labelled datasets a...,/pdf/4651583946d91e7a8825d9046669203387004649.pdf,ICLR,2018,We propose a method of using GANs to generate ...
4,313.000000,708.0,4,B13njo1R-,HkjnjjyR-,1509043124096,1518730186831,162,Progressive Reinforcement Learning with Distil...,"[""gberseth@gmail.com"", ""cheng.k.xie@gmail.com""...","[""Glen Berseth"", ""Cheng Xie"", ""Paul Cernek"", ""...","[""Reinforcement Learning"", ""Distillation"", ""Tr...",Deep reinforcement learning has demonstrated i...,/pdf/71ba098440582330f4a76c53a49ac5abb153f23f.pdf,ICLR,2018,A continual learning method that uses distilla...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2815,1828.500000,5823.0,4,ryzECoAcY7,rkeLkpaqKX,1538087884115,1567618912215,886,Learning Multi-Level Hierarchies with Hindsight,"[""andrew_levy2@brown.edu"", ""gdk@cs.brown.edu"",...","[""Andrew Levy"", ""George Konidaris"", ""Robert Pl...","[""Hierarchical Reinforcement Learning"", ""Reinf...",Hierarchical agents have the potential to solv...,/pdf/30e53961e2765131e7a72310088973ae42f8e70d.pdf,ICLR,2019,We introduce the first Hierarchical RL approac...
2816,4588.600000,12575.0,7,ryzHXnR5Y7,BkxAFx05Y7,1538087964947,1545355407175,1355,Select Via Proxy: Efficient Data Selection For...,"[""cody@cs.stanford.edu"", ""mussmann@stanford.ed...","[""Cody Coleman"", ""Stephen Mussmann"", ""Baharan ...","[""data selection"", ""deep learning"", ""uncertain...","At internet scale, applications collect a trem...",/pdf/4df17e198dea337baaf404031db38b500cf9345c.pdf,ICLR,2019,we develop an efficient method for selecting t...
2817,1395.400000,4194.0,7,ryza73R9tQ,rJl4eu4qtQ,1538087973152,1545355403171,1402,Machine Translation With Weakly Paired Bilingu...,"[""wulijun3@mail2.sysu.edu.cn"", ""teslazhu@mail....","[""Lijun Wu"", ""Jinhua Zhu"", ""Di He"", ""Fei Gao"",...","[""Natural Language Processing"", ""Machine Trans...","Neural machine translation, which achieves nea...",/pdf/e78738172a26113eaf3f8d4f6fa24649c105d430.pdf,ICLR,2019,None
2818,902.000000,902.0,7,ryzfcoR5YQ,SkxO5iocYX,1538087818374,1545355386178,517,Layerwise Recurrent Autoencoder for General Re...,"[""zhaopeize@sensetime.com"", ""caidanfeng@senset...","[""Peize Zhao"", ""Danfeng Cai"", ""Shaokun Zhang"",...","[""traffic flow forecasting"

# Get citation counts/prestige scores

In [54]:
query = '''
SELECT 
    AVG(CAST(reviews.rating_int as demcial)) AS AVG_rating,
    AVG(CAST(reviews.confidence as demcial)) AS AVG_confidence,
    AVG(CAST(review_len.review_len as demcial)) AS AVG_len,
    COUNT(id) as NUM_Reviews,
    forum
FROM reviews
JOIN review_len ON reviews.id = review_len.review_id
GROUP BY forum'''

df_avg_review = pd.read_sql_query(query,con)
df_avg_review.head()

,AVG_rating,AVG_confidence,AVG_len,NUM_Reviews,forum
0,4.5,4.00,441.00,4,--GJkm7nt0
1,6.5,3.75,1316.50,4,--gvHfE3Xf5
2,5.5,4.25,825.75,4,--rcOeCKRh
3,3.5,3.00,683.25,4,-0Cjhnl-dhK
4,4.4,3.80,740.20,5,-0LuSWi6j4


In [55]:
df_submissions = pd.merge(df_submission,df_avg_review, left_on ='id', right_on ='forum', how ='left')
df_submissions

,AVG_CITE,MAX_CITE,AUTHOR_COUNT,id,original,tcdate,tmdate,number,title,authorids,...,abstract,pdf,conf_name,conf_year,tldr,AVG_rating,AVG_confidence,AVG_len,NUM_Reviews,forum
0,1523.500000,2994.0,3,B1-Hhnslg,None,1478376505434,1481060777491,581,Prototypical Networks for Few-shot Learning,"[""jsnell@cs.toronto.edu"", ""kswersky@twitter.co...",...,A recent approach to few-shot classification c...,/pdf/1e1d8a1884188e3ce0db825def307a25c844c868.pdf,ICLR,2017,We learn a metric space in which few-shot clas...,5.000000,4.000000,303.333333,3.0,B1-Hhnslg
1,789.000000,1401.0,2,B1-q5Pqxl,None,1478290057546,1489370977828,417,Machine Comprehension Using Match-LSTM and Ans...,"[""shwang.2014@phdis.smu.edu.sg"", ""jingjiang@sm...",...,Machine comprehension of text is an important ...,/pdf/78402508a0a3ebc1c9078ef8ff48dccc6fef6cd8.pdf,ICLR,2017,Using Match-LSTM and Answer Pointer to select ...,6.333333,3.333333,453.666667,3.0,B1-q5Pqxl
2,168.666667,275.0,3,B12Js_yRb,ryiyj_yRW,1509030627915,1518730187825,136,Learning to Count Objects in Natural Images fo...,"[""yz5n12@ecs.soton.ac.uk"", ""jsh2@ecs.soton.ac....",...,Visual Question Answering (VQA) models have st...,/pdf/b0ec608c19a908ac8ccc2d67f8dc370a699ae26c.pdf,ICLR,2018,Enabling Visual Question Answering models to c...,5.333333,3.333333,509.333333,3.0,B12Js_yRb
3,14.000000,14.0,1,B13EC5u6W,HJh409uTW,1508580916492,1518730191139,36,Thinking like a machine — generating visual ra...,"[""jarrelscy@gmail.com""]",...,Interpretability and small labelled datasets a...,/pdf/4651583946d91e7a8825d9046669203387004649.pdf,ICLR,2018,We propose a method of using GANs to generate ...,6.333333,3.000000,610.666667,3.0,B13EC5u6W
4,313.000000,708.0,4,B13njo1R-,HkjnjjyR-,1509043124096,1518730186831,162,Progressive Reinforcement Learning with Distil...,"[""gberseth@gmail.com"", ""cheng.k.xie@gmail.com""...",...,Deep reinforcement learning has demonstrated i...,/pdf/71ba098440582330f4a76c53a49ac5abb153f23f.pdf,ICLR,2018,A continual learning method that uses distilla...,6.333333,3.666667,502.666667,3.0,B13njo1R-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2815,1828.500000,5823.0,4,ryzECoAcY7,rkeLkpaqKX,1538087884115,1567618912215,886,Learning Multi-Level Hierarchies with Hindsight,"[""andrew_levy2@brown.edu"", ""gdk@cs.brown.edu"",...",...,Hierarchical agents have the potential to solv...,/pdf/30e53961e2765131e7a72310088973ae42f8e70d.pdf,ICLR,2019,We introduce the first Hierarchical RL approac...,6.000000,4.000000,461.000000,3.0,ryzECoAcY7
2816,4588.600000,12575.0,7,ryzHXnR5Y7,BkxAFx05Y7,1538087964947,1545355407175,1355,Select Via Proxy: Efficient Data Selection For...,"[""cody@cs.stanford.edu"", ""mussmann@stanford.ed...",...,"At internet scale, applications collect a trem...",/pdf/4df17e198dea337baaf404031db38b500cf9345c.pdf,ICLR,2019,we develop an efficient method for selecting t...,4.333333,3.333333,535.666667,3.0,ryzHXnR5Y7
2817,1395.400000,4194.0,7,ryza73R9tQ,rJl4eu4qtQ,1538087973152,1545355403171,1402,Machine Translation With Weakly Paired Bilingu...,"[""wulijun3@mail2.sysu.edu.cn"", ""teslazhu@mail....",...,"Neural machine translation, which achieves nea...",/pdf/e78738172a26113eaf3f8d4f6fa24649c105d430.pdf,ICLR,2019,None,6.000000,4.333333,362.000000,3.0,ryza73R9tQ
2818,902.000000,902.0,7,ryzfcoR5YQ,SkxO5iocYX,1538087818374,1545355386178,517,Layerwise Recurrent Autoencoder for General Re...,"[""zhaopeize@sensetime.com"", ""caidanfeng@senset...",...,Accurate spatio-temporal traffic forecasting i...,/pdf/819fe626b59be1697c22ab5d9570defb54e49754.pdf,ICLR,2019,We propose Layerwise Recurrent Autoencoder wit...,4.000000,3.333333,421.333333,3.0,ryzfcoR5YQ


In [56]:
df_comments = df_submission_features[df_submission_features.comment.notnull()]
df_review = df_submission_features[df_submission_features.review.notnull()]

df_review = df_review.groupby(['forum'],as_index=False )['polite_prediction'].mean()

df_comments = df_comments.groupby(['forum'],as_index=False ).mean()

In [57]:
df_submissions = pd.merge(df_submissions,df_review, left_on ='id', right_on ='forum', how ='left')

In [58]:
df_submissions

,AVG_CITE,MAX_CITE,AUTHOR_COUNT,id,original,tcdate,tmdate,number,title,authorids,...,conf_name,conf_year,tldr,AVG_rating,AVG_confidence,AVG_len,NUM_Reviews,forum_x,forum_y,polite_prediction
0,1523.500000,2994.0,3,B1-Hhnslg,None,1478376505434,1481060777491,581,Prototypical Networks for Few-shot Learning,"[""jsnell@cs.toronto.edu"", ""kswersky@twitter.co...",...,ICLR,2017,We learn a metric space in which few-shot clas...,5.000000,4.000000,303.333333,3.0,B1-Hhnslg,B1-Hhnslg,0.666667
1,789.000000,1401.0,2,B1-q5Pqxl,None,1478290057546,1489370977828,417,Machine Comprehension Using Match-LSTM and Ans...,"[""shwang.2014@phdis.smu.edu.sg"", ""jingjiang@sm...",...,ICLR,2017,Using Match-LSTM and Answer Pointer to select ...,6.333333,3.333333,453.666667,3.0,B1-q5Pqxl,B1-q5Pqxl,0.000000
2,168.666667,275.0,3,B12Js_yRb,ryiyj_yRW,1509030627915,1518730187825,136,Learning to Count Objects in Natural Images fo...,"[""yz5n12@ecs.soton.ac.uk"", ""jsh2@ecs.soton.ac....",...,ICLR,2018,Enabling Visual Question Answering models to c...,5.333333,3.333333,509.333333,3.0,B12Js_yRb,B12Js_yRb,0.333333
3,14.000000,14.0,1,B13EC5u6W,HJh409uTW,1508580916492,1518730191139,36,Thinking like a machine — generating visual ra...,"[""jarrelscy@gmail.com""]",...,ICLR,2018,We propose a method of using GANs to generate ...,6.333333,3.000000,610.666667,3.0,B13EC5u6W,B13EC5u6W,0.333333
4,313.000000,708.0,4,B13njo1R-,HkjnjjyR-,1509043124096,1518730186831,162,Progressive Reinforcement Learning with Distil...,"[""gberseth@gmail.com"", ""cheng.k.xie@gmail.com""...",...,ICLR,2018,A continual learning method that uses distilla...,6.333333,3.666667,502.666667,3.0,B13njo1R-,B13njo1R-,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2815,1828.500000,5823.0,4,ryzECoAcY7,rkeLkpaqKX,1538087884115,1567618912215,886,Learning Multi-Level Hierarchies with Hindsight,"[""andrew_levy2@brown.edu"", ""gdk@cs.brown.edu"",...",...,ICLR,2019,We introduce the first Hierarchical RL approac...,6.000000,4.000000,461.000000,3.0,ryzECoAcY7,ryzECoAcY7,0.333333
2816,4588.600000,12575.0,7,ryzHXnR5Y7,BkxAFx05Y7,1538087964947,1545355407175,1355,Select Via Proxy: Efficient Data Selection For...,"[""cody@cs.stanford.edu"", ""mussmann@stanford.ed...",...,ICLR,2019,we develop an efficient method for selecting t...,4.333333,3.333333,535.666667,3.0,ryzHXnR5Y7,ryzHXnR5Y7,1.000000
2817,1395.400000,4194.0,7,ryza73R9tQ,rJl4eu4qtQ,1538087973152,1545355403171,1402,Machine Translation With Weakly Paired Bilingu...,"[""wulijun3@mail2.sysu.edu.cn"", ""teslazhu@mail....",...,ICLR,2019,None,6.000000,4.333333,362.000000,3.0,ryza73R9tQ,ryza73R9tQ,0.333333
2818,902.000000,902.0,7,ryzfcoR5YQ,SkxO5iocYX,1538087818374,1545355386178,517,Layerwise Recurrent Autoencoder for General Re...,"[""zhaopeize@sensetime.com"", ""caidanfeng@senset...",...,ICLR,2019,We propose Layerwise Recurrent Autoencoder wit...,4.000000,3.333333,421.333333,3.0,ryzfcoR5YQ,ryzfcoR5YQ,0.333333


# Get Location, US/Canada

In [59]:
tld_dct = pd.read_csv("https://gist.githubusercontent.com/derlin/421d2bb55018a1538271227ff6b1299d/raw/3a131d47ca322a1d001f1f79333d924672194f36/country-codes-tlds.csv")
tld_dct[' tld'] = tld_dct[' tld'].apply(lambda x: x[-3:])
tld_dct[' tld'] = tld_dct[' tld'].apply(lambda x: x.strip("."))
tld_dct = dict(zip(tld_dct[' tld'], tld_dct['country']))
tld_dct['edu'] = "United States"
tld_dct['gov'] = "United States"

In [60]:
query = '''
SELECT author_submission.*, 
submissions.conf_year,
author_institution.*,
authors.name

FROM
author_submission
LEFT JOIN author_institution ON author_submission.author_id = author_institution.author_id
JOIN submissions ON author_submission.submission_id = submissions.id
JOIN authors ON author_submission.author_id = authors.id
WHERE submissions.conf_year in (2017,2018,2019)
'''

df_authors = pd.read_sql_query(query,con)

In [61]:
df_authors.head()

,author_id,submission_id,author_no,conf_year,position,ins_domain,ins_name,start,end,author_id,institution_id,ins_root,tld,name
0,lixin@eecs.oregonstate.edu,S1TER2oll,1,2017,None,None,None,None,None,None,None,None,None,Xingyi Li
1,~Fuxin_Li1,S1TER2oll,2,2017,Assistant Professor,oregonstate.edu,Oregon State University,2015,None,~Fuxin_Li1,f9d00c4a,Oregon State University,edu,Li Fuxin
2,~Xiaoli_Fern1,S1TER2oll,3,2017,Associate Professor,oregonstate.edu,Oregon State University,2005,None,~Xiaoli_Fern1,f9d00c4a,Oregon State University,edu,Xiaoli Fern
3,~Raviv_Raich1,S1TER2oll,4,2017,Associate Professor,oregonstate.edu,Oregon State University,2016,None,~Raviv_Raich1,f9d00c4a,Oregon State University,edu,Raviv Raich
4,~Danica_J._Sutherland2,HJWHIKqgl,1,2017,Postdoc,ucl.ac.uk,"Gatsby Computational Neuroscience Unit, Univer...",2016,2019,~Danica_J._Sutherland2,None,"Gatsby Computational Neuroscience Unit, Univer...",uk,Danica J. Sutherland


In [62]:
tld_dct['edu']

'United States'

In [63]:
df_reference_emails = pd.read_csv('../data/authors/reference_data/coded_missing_email_locations.csv')


In [64]:

def get_country(s, tld_dct): 
    """
    Takes TLD, returns country.
    """
    if s.tld in tld_dct:
        country = tld_dct[s.tld]
        if country in ('United States', 'Canada'):
            return 1
        else:
            return 0
    else:
        return None
    

df_authors['US_Canada'] = df_authors.apply(lambda x: get_country(x, tld_dct), axis = 1)


In [65]:
def lambda_get_submission_year_position(row):
    try:
        if row.start is None:
            return None
        else:
            start = pd.to_datetime(row.start,format='%Y')
        if row.end is None:
            end = pd.Timestamp.now()
        else:
            if float(row.end) >= 2023:
                end= pd.Timestamp.now()
            else:
                end = pd.to_datetime(row.end,format='%Y')
        dict_conf_year = {'2017':pd.to_datetime('2/6/2017'),'2018':pd.to_datetime('1/29/2018'),'2019':pd.to_datetime('12/22/2018')}
        if start <= dict_conf_year[row.conf_year] < end:
            return 1
        else:
            return 0
    except:
        print('row.start',row.start, 'row.end',row.end, 'start',start, 'end',end,'year', row.conf_year)
df_authors['current_position_flag'] = df_authors.apply(lambda x: lambda_get_submission_year_position(x), axis = 1)    

In [66]:
def lambda_clean_position(row):
    position =row.position
    if position is None:
        return 'Unknown'
    else:
        try:
            position = position.lower().strip('.').strip('/')
        except:
            return 'Unknown'
    if 'professor' in  position:
        return 'professor'
    elif 'student' in position or 'phd' in position:
        return 'student'
    elif 'post' in position:
        return 'postdoc'
    else:
        return 'other'
df_authors['clean_position'] = df_authors.apply(lambda x: lambda_clean_position(x), axis = 1)

In [67]:
df_authors.position.unique()

array([None, 'Assistant Professor', 'Associate Professor', 'Postdoc',
       'PhD student', 'Research Assistant Professor', 'CO-I',
       'Research Scientist', 'Distinguished Scientist', 'Full Professor',
       'Adjunct Professor', 'Principal Research Scientist', 'Professor',
       'MS student', 'Visiting Scientist', 'Stanford University',
       'Principal Program Manager Lead', 'Researcher', 'Engineer',
       'Partner Research Manager', 'Undergrad student',
       'Visiting research engineer', 'Visiting researcher', 'Intern',
       'Software Engineer', 'Staff Research Scientist', 'Scientist',
       'Senior Research Scientist', 'Graduate Student', 'Chief Scientist',
       'Machine Learning Research Scientist', 'Research Asistant',
       'researcher', 'Employee', 'Research scientist',
       'Research director', 'Senior Staff Research Scientist',
       'Staff Research Engineer', 'Associate professor',
       'Member of Technical Staff', 'Founder', 'research scientist',
       

In [68]:
df_authors['clean_position']

0          Unknown
1        professor
2        professor
3        professor
4          postdoc
           ...    
24168      Unknown
24169      Unknown
24170      student
24171      Unknown
24172    professor
Name: clean_position, Length: 24173, dtype: object

# Get percieved gender 

In [69]:
name_woman

{'\x06': [None, None],
 'Pooya': ['man', 0.010000000000000009],
 'Zhengyang': ['man', 0.040000000000000036],
 'Kishore': ['man', 0.0],
 'Visvanathan': ['man', 0.050000000000000044],
 'Zhaoyuan': ['man', 0.27],
 'Zhenzhong': ['man', 0.050000000000000044],
 'Howie': ['man', 0.0],
 'JIANXIONG': ['man', 0.0],
 'Jim': ['man', 0.0],
 'Venu': ['man', 0.030000000000000027],
 'Sayantan': ['man', 0.07999999999999996],
 'Mengmi': ['man', 0.0],
 'Keng': ['man', 0.030000000000000027],
 'Shih-Cheng': ['man', 0.0],
 'Gintare': ['woman', 0.98],
 'Zana': ['woman', 0.85],
 'Mehran': ['man', 0.0],
 'Seyyed': ['man', 0.06000000000000005],
 'Javad': ['man', 0.0],
 'Jarrel': ['man', 0.0],
 'Yuanhang': ['man', 0.12],
 'Yuzhong': ['man', 0.07999999999999996],
 'C.-C.': ['man', 0.0],
 'Bence': ['man', 0.0],
 'Dimche': ['man', 0.0],
 'Slava': ['man', 0.08999999999999997],
 'Vitor': ['man', 0.0],
 'Olga': ['woman', 0.99],
 'Luping': ['woman', 0.56],
 'Sivan': ['man', 0.49],
 'Björn': ['man', 0.0],
 'chao': ['man

In [70]:
first_name = 'Xingyi'
if first_name in data.keys():
    print('test')

NameError: name 'data' is not defined

In [71]:
with open('name_woman.json', 'r') as file:
    name_woman = json.load(file)

In [72]:
#run if need to update the names dictionary
'''
genderize = Genderize(api_key = 'bbe489da604ffbc66a8b9f0764dd88ce')

name_woman = {}
for name in tqdm.tqdm(df_authors['name'].unique()):
    first_name = name.split(' ')[0]
    if first_name in data.keys():
        pass
    else:
        try:
            if first_name:
                sex = genderize.get([first_name])[0]['gender']
                prob_woman = genderize.get([first_name])[0]['probability']
                if sex == 'female':
                    gender = 'woman'
                elif sex == 'male': 
                    gender = 'man'
                    prob_woman = 1 - prob_woman 
                name_woman[first_name] = (gender, prob_woman)
            else: 
                name_woman[first_name] = (None, None)
        except:
            name_woman[first_name] = (None, None)
'''

"\ngenderize = Genderize(api_key = 'bbe489da604ffbc66a8b9f0764dd88ce')\n\nname_woman = {}\nfor name in tqdm.tqdm(df_authors['name'].unique()):\n    first_name = name.split(' ')[0]\n    if first_name in data.keys():\n        pass\n    else:\n        try:\n            if first_name:\n                sex = genderize.get([first_name])[0]['gender']\n                prob_woman = genderize.get([first_name])[0]['probability']\n                if sex == 'female':\n                    gender = 'woman'\n                elif sex == 'male': \n                    gender = 'man'\n                    prob_woman = 1 - prob_woman \n                name_woman[first_name] = (gender, prob_woman)\n            else: \n                name_woman[first_name] = (None, None)\n        except:\n            name_woman[first_name] = (None, None)\n"

In [73]:
tqdm.tqdm.pandas()
def get_gender(row):
    first_name = row['name'].split(' ')[0]
    try:
        return name_woman[first_name][0], name_woman[first_name][1]
    except:
        sex = genderize.get([first_name])[0]['gender']
        prob_woman = genderize.get([first_name])[0]['probability']
        print(sex)
        if sex == 'female':
            gender = 'woman'
        elif sex == 'male': 
            gender = 'man'
            prob_woman = 1 - prob_woman 
        return gender, prob_woman


df_authors[['gender','prob_women']] = df_authors.progress_apply(lambda x: get_gender(x), axis = 1,result_type='expand')

df_authors

100%|████████████████████████████████| 24173/24173 [00:00<00:00, 66557.33it/s]


,author_id,submission_id,author_no,conf_year,position,ins_domain,ins_name,start,end,author_id,institution_id,ins_root,tld,name,US_Canada,current_position_flag,clean_position,gender,prob_women
0,lixin@eecs.oregonstate.edu,S1TER2oll,1,2017,None,None,None,None,None,None,None,None,None,Xingyi Li,NaN,NaN,Unknown,woman,0.83
1,~Fuxin_Li1,S1TER2oll,2,2017,Assistant Professor,oregonstate.edu,Oregon State University,2015,None,~Fuxin_Li1,f9d00c4a,Oregon State University,edu,Li Fuxin,1.0,1.0,professor,woman,0.66
2,~Xiaoli_Fern1,S1TER2oll,3,2017,Associate Professor,oregonstate.edu,Oregon State University,2005,None,~Xiaoli_Fern1,f9d00c4a,Oregon State University,edu,Xiaoli Fern,1.0,1.0,professor,woman,0.88
3,~Raviv_Raich1,S1TER2oll,4,2017,Associate Professor,oregonstate.edu,Oregon State University,2016,None,~Raviv_Raich1,f9d00c4a,Oregon State University,edu,Raviv Raich,1.0,1.0,professor,man,0.00
4,~Danica_J._Sutherland2,HJWHIKqgl,1,2017,Postdoc,ucl.ac.uk,"Gatsby Computational Neuroscience Unit, Univer...",2016,2019,~Danica_J._Sutherland2,None,"Gatsby Computational Neuroscience Unit, Univer...",uk,Danica J. Sutherland,0.0,1.0,postdoc,woman,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24168,xptree@gmail.com,S1sRrN-CW,1,2018,None,None,None,None,None,None,None,None,None,Jiezhong Qiu,NaN,NaN,Unknown,man,0.00
24169,~Hao_Ma2,S1sRrN-CW,2,2018,None,None,None,None,None,None,None,None,None,Hao Ma,NaN,NaN,Unknown,man,0.10
24170,~Yuxiao_Dong2,S1sRrN-CW,3,2018,PhD student,nd.edu,University of Notre Dame,2012,2017,~Yuxiao_Dong2,a21164a7,University of Notre Dame,edu,Yuxiao Dong,1.0,0.0,student,woman,0.67
24171,~Kuansan_Wang1,S1sRrN-CW,4,2018,None,None,None,None,None,None,None,None,None,Kuansan Wang,NaN,NaN,Unknown,man,0.00


In [74]:
df_authors['position'].unique()

array([None, 'Assistant Professor', 'Associate Professor', 'Postdoc',
       'PhD student', 'Research Assistant Professor', 'CO-I',
       'Research Scientist', 'Distinguished Scientist', 'Full Professor',
       'Adjunct Professor', 'Principal Research Scientist', 'Professor',
       'MS student', 'Visiting Scientist', 'Stanford University',
       'Principal Program Manager Lead', 'Researcher', 'Engineer',
       'Partner Research Manager', 'Undergrad student',
       'Visiting research engineer', 'Visiting researcher', 'Intern',
       'Software Engineer', 'Staff Research Scientist', 'Scientist',
       'Senior Research Scientist', 'Graduate Student', 'Chief Scientist',
       'Machine Learning Research Scientist', 'Research Asistant',
       'researcher', 'Employee', 'Research scientist',
       'Research director', 'Senior Staff Research Scientist',
       'Staff Research Engineer', 'Associate professor',
       'Member of Technical Staff', 'Founder', 'research scientist',
       

# Save outputs 

In [75]:
df_authors.to_csv('../data/database/outputs/df_authors.csv',index=False)

In [76]:
df_submissions.to_csv('../data/database/outputs/df_prestige.csv',index=False)

In [77]:
#with open("name_woman.json", "w") as outfile:
#    json.dump(name_woman, outfile)